### Finviz Data Cleaning and Transformation Pipeline

This notebook processes the raw data downloaded from Finviz, cleans it, and transforms it into an analysis-ready format.

**Workflow:**

1.  **Load Data:** Reads a raw Finviz `.parquet` file for a specific date.
2.  **Feature Engineering:** Creates new composite columns (`Info`, `MktCap AUM`).
3.  **Data Type Conversion:**
    *   Converts currency strings (e.g., `1.5B`, `250K`) into numeric values in millions.
    *   Converts percentage strings (e.g., `12.5%`) into numeric values.
    *   Converts other object columns to their proper numeric types.
4.  **Final Processing:** Sorts the data by market capitalization, sets the `Ticker` as the index, and adds a `Rank` column.
5.  **Save & Verify:** Saves the cleaned DataFrame to a new `.parquet` file and verifies the saved file.

### Setup and Configuration

**This is the only cell you need to modify.** It contains all imports, paths, and lists of columns for processing.

In [1]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np

# --- Project Path Setup ---
NOTEBOOK_DIR = Path.cwd()
ROOT_DIR = NOTEBOOK_DIR.parent if NOTEBOOK_DIR.name == 'notebooks' else NOTEBOOK_DIR
if str(ROOT_DIR) not in sys.path:
    sys.path.append(str(ROOT_DIR))

SRC_DIR = ROOT_DIR / 'src'
if str(SRC_DIR) not in sys.path:
    sys.path.append(str(SRC_DIR))

# Import config and custom utils now that path is set
from config import DATE_STR, DOWNLOAD_DIR, DEST_DIR
import utils

# --- File Path Configuration ---
# Build paths using pathlib for cross-platform compatibility
SOURCE_PATH = Path(DOWNLOAD_DIR) / f'df_finviz_{DATE_STR}_stocks_etfs.parquet'
DEST_PATH = Path(DEST_DIR) / f'{DATE_STR}_df_finviz_stocks_etfs.parquet'

# --- Column Processing Configuration ---
# Define which columns need specific cleaning operations.

# Columns to combine into the 'Info' column
INFO_COLS = ["Sector", "Industry", "Single Category", "Asset Type"]

# Columns with abbreviated currency values (B, M, K) to be converted to millions
CURRENCY_COLS = [
    'Market Cap', 'AUM', 'Sales', 'Income', 'Outstanding', 'Float', 
    'Short Interest', 'Avg Volume', 'Flows 1M', 'Flows 3M', 'Flows YTD',
    'MktCap AUM' # This is the new column we create
]

# Other columns that are numeric but stored as strings (objects)
# Note: Percentage columns are detected automatically in Step 3.
OTHER_NUMERIC_COLS = [
    "No.", "P/E", "Fwd P/E", "PEG", "P/S", "P/B", "P/C", "P/FCF",
    "Book/sh", "Cash/sh", "Dividend TTM", "EPS", "EPS next Q", "Short Ratio",
    "Curr R", "Quick R", "LTDebt/Eq", "Debt/Eq", "Beta", "ATR", "RSI",
    "Employees", "Recom", "Rel Volume", "Volume", "Target Price",
    "Prev Close", "Open", "High", "Low", "Price", "Holdings"
]

# --- Notebook Setup ---
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)
pd.set_option('display.width', 2500)
%load_ext autoreload
%autoreload 2

# --- Verification ---
print(f"Source file: {SOURCE_PATH}")
print(f"Destination file: {DEST_PATH}")
print(f"Processing for date: {DATE_STR}")

Source file: C:\Users\ping\Downloads\df_finviz_2025-07-01_stocks_etfs.parquet
Destination file: c:\Users\ping\Files_win10\python\py311\stocks_v0_works\data\2025-07-01_df_finviz_stocks_etfs.parquet
Processing for date: 2025-07-01


### Step 1: Load Raw Data

Load the source Parquet file into a pandas DataFrame.

In [2]:
print(f"--- Step 1: Loading data from {SOURCE_PATH.name} ---")

try:
    df = pd.read_parquet(SOURCE_PATH, engine='pyarrow')
    print("Data loaded successfully.")
    df.info()
    display(df.head(3))
except FileNotFoundError:
    print(f"ERROR: Source file not found at {SOURCE_PATH}")
    df = None  # Ensure df is None if loading fails
except Exception as e:
    print(f"An error occurred during file loading: {e}")
    df = None

--- Step 1: Loading data from df_finviz_2025-07-01_stocks_etfs.parquet ---
Data loaded successfully.


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Columns: 111 entries, No. to Tags
dtypes: object(111)
memory usage: 1.3+ MB


,No.,Ticker,Company,Index,Sector,Industry,Country,Exchange,Market Cap,P/E,Fwd P/E,PEG,P/S,P/B,P/C,P/FCF,Book/sh,Cash/sh,Dividend,Dividend TTM,Dividend Ex Date,Payout Ratio,EPS,EPS next Q,EPS This Y,EPS Next Y,EPS Past 5Y,EPS Next 5Y,Sales Past 5Y,Sales Q/Q,EPS Q/Q,EPS YoY TTM,Sales YoY TTM,Sales,Income,EPS Surprise,Revenue Surprise,Outstanding,Float,Float %,Insider Own,Insider Trans,Inst Own,Inst Trans,Short Float,Short Ratio,Short Interest,ROA,ROE,ROIC,Curr R,Quick R,LTDebt/Eq,Debt/Eq,Gross M,Oper M,Profit M,Perf Week,Perf Month,Perf Quart,Perf Half,Perf Year,Perf YTD,Beta,ATR,Volatility W,Volatility M,SMA20,SMA50,SMA200,50D High,50D Low,52W High,52W Low,52W Range,All-Time High,All-Time Low,RSI,Earnings,IPO Date,Optionable,Shortable,Employees,Change from Open,Gap,Recom,Avg Volume,Rel Volume,Volume,Target Price,Prev Close,Open,High,Low,Price,Change,Single Category,Asset Type,Expense,Holdings,AUM,Flows 1M,Flows% 1M,Flows 3M,Flows% 3M,Flows YTD,Flows% YTD,Return% 1Y,Return% 3Y,Return% 5Y,Tags
0,801,LECO,"Lincoln Electric Holdings, Inc",-,Industrials,Tools & Accessories,USA,NASD,11.74B,25.94,21.12,6.52,2.91,8.76,29.73,21.97,24.01,7.07,1.43%,2.96,6/30/2025,35.34%,8.10,2.30,-2.74%,10.15%,11.74%,3.98%,5.96%,2.52%,-2.10%,-14.49%,-2.29%,4.03B,461.18M,-3.18%,2.50%,55.83M,53.34M,95.54%,4.45%,-1.54%,78.75%,-1.12%,2.12%,3.02,1.13M,13.17%,34.83%,18.23%,1.78,1.19,0.89,0.98,36.60%,17.48%,11.43%,1.99%,8.59%,11.13%,10.13%,12.35%,12.14%,1.21,4.87,2.66%,2.14%,3.27%,7.97%,7.35%,-1.94%,23.88%,-5.53%,30.48%,161.11 - 222.52,-19.50%,3904.19%,63.02,Apr 30/b,6/13/1995,Yes,Yes,12000,1.68%,-0.27%,2.27,373.75K,1.35,"504,587",212.12,207.32,206.75,214.57,205.36,210.22,1.40%,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
1,802,SCI,Service Corp. International,-,Consumer Cyclical,Personal Services,USA,NYSE,11.70B,22.64,19.68,2.13,2.77,7.13,49.70,18.29,11.52,1.65,2.97%,1.24,6/13/2025,33.96%,3.63,0.85,7.22%,10.29%,12.14%,10.65%,5.32%,2.75%,10.78%,4.07%,2.40%,4.22B,530.23M,5.73%,1.25%,143.27M,138.20M,96.46%,2.92%,-1.47%,89.56%,-4.12%,4.13%,4.75,5.71M,3.12%,32.71%,8.30%,0.51,0.46,2.87,2.92,26.28%,22.90%,12.58%,3.22%,5.33%,2.44%,1.42%,14.51%,2.93%,0.93,1.29,1.65%,1.45%,3.36%,4.97%,3.27%,0.26%,10.82%,-8.07%,19.35%,68.84 - 89.37,-8.07%,5158.24%,68.57,Apr 30/a,12/7/1970,Yes,Yes,24953,0.96%,-0.02%,1.33,1.20M,0.84,"1,009,225",89.80,81.40,81.38,83.04,81.14,82.16,0.93%,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
2,803,AR,Antero Resources Corp,-,Energy,Oil & Gas E&P,USA,NYSE,11.69B,52.00,9.17,0.44,2.52,1.63,-,12.36,23.17,0.00,-,-,-,0.00%,0.72,0.56,796.43%,31.66%,-,119.43%,1.86%,28.00%,465.10%,240.70%,6.67%,4.64B,228.85M,-6.11%,-3.16%,311.58M,288.75M,92.67%,7.01%,-8.65%,86.57%,2.04%,4.68%,2.64,13.53M,1.72%,3.21%,2.18%,0.39,0.39,0.46,0.53,12.35%,7.25%,4.94%,-11.07%,0.56%,-6.87%,14.78%,15.45%,7.45%,0.69,1.42,3.29%,3.37%,-6.66%,-2.39%,7.88%,-14.44%,20.05%,-14.44%,53.53%,24.53 - 44.01,-44.97%,5802.82%,39.00,Apr 30/a,10/10/2013,Yes,Yes,616,-5.83%,-0.72%,1.92,5.12M,1.76,"9,036,113",46.43,40.28,39.99,40.05,37.65,37.66,-6.50%,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-


### Step 2: Feature Engineering - Create Composite Columns

Combine existing columns to create more meaningful features: `Info` and `MktCap AUM`.

In [3]:
if df is not None:
    print("\n--- Step 2: Engineering new features ---")
    
    # 1. Create 'Info' column by concatenating category columns.
    for col in INFO_COLS:
        if col in df.columns:
            df[col] = df[col].replace('-', '')
    df['Info'] = df[INFO_COLS].apply(lambda row: ', '.join(filter(None, row.astype(str))), axis=1)
    print("Created 'Info' column.")

    # 2. Create 'MktCap AUM' by concatenating 'Market Cap' and 'AUM'.
    # This combines stock and ETF liquidity metrics into a single string column for now.
    # It will be converted to numeric in the next step.
    df['MktCap AUM'] = df['Market Cap'].replace('-', '') + df['AUM'].replace('-', '')
    print("Created 'MktCap AUM' column.")

    # Display the new columns for verification
    display(df[['Ticker', 'Info', 'MktCap AUM']].head(3))


--- Step 2: Engineering new features ---


Created 'Info' column.
Created 'MktCap AUM' column.


,Ticker,Info,MktCap AUM
0,LECO,"Industrials, Tools & Accessories",11.74B
1,SCI,"Consumer Cyclical, Personal Services",11.70B
2,AR,"Energy, Oil & Gas E&P",11.69B


### Step 3: Data Type Conversion

This multi-part step cleans and converts all string-based numeric and percentage columns into proper numeric types.

#### Part A: Convert Abbreviated Currency Columns to Millions

In [4]:
def convert_to_millions(value: str) -> float:
    """Converts a string with a T/B/M/K suffix to a numeric value in millions."""
    if pd.isna(value):
        return np.nan
    
    value_str = str(value).strip().upper()
    if not value_str:
        return np.nan

    multipliers = {'T': 1_000_000, 'B': 1_000, 'M': 1, 'K': 0.001}
    suffix = value_str[-1]
    
    if suffix in multipliers:
        number_part = value_str[:-1]
        try:
            return float(number_part) * multipliers[suffix]
        except (ValueError, TypeError):
            return np.nan
    return np.nan

if df is not None:
    print("\n--- Step 3a: Converting currency columns to millions ---")
    new_names = {}
    for col in CURRENCY_COLS:
        if col in df.columns:
            df[col] = df[col].apply(convert_to_millions)
            new_names[col] = f"{col}, M"
    
    df.rename(columns=new_names, inplace=True)
    print(f"Converted and renamed {len(new_names)} columns.")
    display(df[[name for name in new_names.values() if name in df.columns]].head(3))


--- Step 3a: Converting currency columns to millions ---
Converted and renamed 12 columns.


,"Market Cap, M","AUM, M","Sales, M","Income, M","Outstanding, M","Float, M","Short Interest, M","Avg Volume, M","Flows 1M, M","Flows 3M, M","Flows YTD, M","MktCap AUM, M"
0,11740.0,NaN,4030.0,461.18,55.83,53.34,1.13,0.37375,NaN,NaN,NaN,11740.0
1,11700.0,NaN,4220.0,530.23,143.27,138.20,5.71,1.20000,NaN,NaN,NaN,11700.0
2,11690.0,NaN,4640.0,228.85,311.58,288.75,13.53,5.12000,NaN,NaN,NaN,11690.0


#### Part B: Convert Percentage Columns to Numeric

In [5]:
if df is not None:
    print("\n--- Step 3b: Converting percentage columns ---")
    percent_cols = [
        col for col in df.columns if df[col].dtype == 'object' and df[col].str.endswith('%', na=False).any()
    ]

    if not percent_cols:
        print("No new percentage columns found to modify.")
    else:
        print("Processing the following percentage columns:")
        for col in percent_cols:
            df[col] = pd.to_numeric(df[col].str.replace('%', ''), errors='coerce')
            new_name = f"{col} %" if '%' not in col else col
            df.rename(columns={col: new_name}, inplace=True)
            print(f"  - Converted '{col}' to numeric and renamed to '{new_name}'")


--- Step 3b: Converting percentage columns ---


Processing the following percentage columns:
  - Converted 'Dividend' to numeric and renamed to 'Dividend %'
  - Converted 'Payout Ratio' to numeric and renamed to 'Payout Ratio %'
  - Converted 'EPS This Y' to numeric and renamed to 'EPS This Y %'
  - Converted 'EPS Next Y' to numeric and renamed to 'EPS Next Y %'
  - Converted 'EPS Past 5Y' to numeric and renamed to 'EPS Past 5Y %'
  - Converted 'EPS Next 5Y' to numeric and renamed to 'EPS Next 5Y %'
  - Converted 'Sales Past 5Y' to numeric and renamed to 'Sales Past 5Y %'
  - Converted 'Sales Q/Q' to numeric and renamed to 'Sales Q/Q %'
  - Converted 'EPS Q/Q' to numeric and renamed to 'EPS Q/Q %'
  - Converted 'EPS YoY TTM' to numeric and renamed to 'EPS YoY TTM %'
  - Converted 'Sales YoY TTM' to numeric and renamed to 'Sales YoY TTM %'
  - Converted 'EPS Surprise' to numeric and renamed to 'EPS Surprise %'
  - Converted 'Revenue Surprise' to numeric and renamed to 'Revenue Surprise %'
  - Converted 'Float %' to numeric and rename

  - Converted '52W Low' to numeric and renamed to '52W Low %'
  - Converted 'All-Time High' to numeric and renamed to 'All-Time High %'
  - Converted 'All-Time Low' to numeric and renamed to 'All-Time Low %'
  - Converted 'Change from Open' to numeric and renamed to 'Change from Open %'
  - Converted 'Gap' to numeric and renamed to 'Gap %'
  - Converted 'Change' to numeric and renamed to 'Change %'
  - Converted 'Expense' to numeric and renamed to 'Expense %'
  - Converted 'Flows% 1M' to numeric and renamed to 'Flows% 1M'
  - Converted 'Flows% 3M' to numeric and renamed to 'Flows% 3M'
  - Converted 'Flows% YTD' to numeric and renamed to 'Flows% YTD'
  - Converted 'Return% 1Y' to numeric and renamed to 'Return% 1Y'
  - Converted 'Return% 3Y' to numeric and renamed to 'Return% 3Y'
  - Converted 'Return% 5Y' to numeric and renamed to 'Return% 5Y'


#### Part C: Convert Other String-Based Numeric Columns

In [6]:
if df is not None:
    print("\n--- Step 3c: Converting other numeric string columns ---")
    converted_count = 0
    for col in OTHER_NUMERIC_COLS:
        if col in df.columns and df[col].dtype == 'object':
            df[col] = pd.to_numeric(df[col].str.replace(',', '', regex=False), errors='coerce')
            converted_count += 1
            
    print(f"Converted {converted_count} additional columns to numeric type.")
    print("\nData types after all conversions:")
    df.info()


--- Step 3c: Converting other numeric string columns ---
Converted 32 additional columns to numeric type.

Data types after all conversions:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Columns: 113 entries, No. to MktCap AUM, M
dtypes: float64(94), int64(2), object(17)
memory usage: 1.3+ MB


### Step 4: Final Processing - Sort, Index, and Rank

Sort the DataFrame by the unified liquidity metric, set the `Ticker` as the index, and add a final `Rank`.

In [7]:
if df is not None:
    print("\n--- Step 4: Finalizing DataFrame ---")
    
    # 1. Sort by the primary metric in descending order
    df.sort_values(by='MktCap AUM, M', ascending=False, inplace=True, na_position='last')
    print("Sorted DataFrame by 'MktCap AUM, M'.")
    
    # 2. Add a 'Rank' column based on the new sort order
    df['Rank'] = range(1, len(df) + 1)
    print("Added 'Rank' column.")
    
    # 3. Set 'Ticker' as the index
    if 'Ticker' in df.columns:
        df.set_index('Ticker', inplace=True)
        print("Set 'Ticker' as the index.")
    
    print("\nFinal DataFrame structure:")
    display(df[['Rank', 'Info', 'MktCap AUM, M']].head())


--- Step 4: Finalizing DataFrame ---
Sorted DataFrame by 'MktCap AUM, M'.
Added 'Rank' column.
Set 'Ticker' as the index.

Final DataFrame structure:


C:\Users\ping\AppData\Local\Temp\ipykernel_8920\3337454481.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['Rank'] = range(1, len(df) + 1)


,Rank,Info,"MktCap AUM, M"
Ticker,,,
NVDA,1,"Technology, Semiconductors",3740520.0
MSFT,2,"Technology, Software - Infrastructure",3657180.0
AAPL,3,"Technology, Consumer Electronics",3103960.0
AMZN,4,"Consumer Cyclical, Internet Retail",2340480.0
GOOGL,5,"Communication Services, Internet Content & Inf...",2140140.0


### Step 5: Save and Verify Cleaned Data

Save the final, cleaned DataFrame to a new Parquet file and read it back to verify integrity.

In [8]:
if df is not None:
    print("\n--- Step 5: Saving and verifying data ---")
    try:
        # Ensure destination directory exists
        DEST_PATH.parent.mkdir(parents=True, exist_ok=True)
        
        # Save the file
        df.to_parquet(DEST_PATH, engine='pyarrow', compression='zstd')
        print(f"Successfully saved cleaned data to: {DEST_PATH}")

        # Verify by loading it back
        loaded_df = pd.read_parquet(DEST_PATH, engine='pyarrow')
        print("\nVerification successful. First 20 rows of the saved file:")
        display(loaded_df.head(20))
        
    except Exception as e:
        print(f"An error occurred during save or verification: {e}")


--- Step 5: Saving and verifying data ---
Successfully saved cleaned data to: c:\Users\ping\Files_win10\python\py311\stocks_v0_works\data\2025-07-01_df_finviz_stocks_etfs.parquet



Verification successful. First 20 rows of the saved file:


,No.,Company,Index,Sector,Industry,Country,Exchange,"Market Cap, M",P/E,Fwd P/E,PEG,P/S,P/B,P/C,P/FCF,Book/sh,Cash/sh,Dividend %,Dividend TTM,Dividend Ex Date,Payout Ratio %,EPS,EPS next Q,EPS This Y %,EPS Next Y %,EPS Past 5Y %,EPS Next 5Y %,Sales Past 5Y %,Sales Q/Q %,EPS Q/Q %,EPS YoY TTM %,Sales YoY TTM %,"Sales, M","Income, M",EPS Surprise %,Revenue Surprise %,"Outstanding, M","Float, M",Float %,Insider Own %,Insider Trans %,Inst Own %,Inst Trans %,Short Float %,Short Ratio,"Short Interest, M",ROA %,ROE %,ROIC %,Curr R,Quick R,LTDebt/Eq,Debt/Eq,Gross M %,Oper M %,Profit M %,Perf Week %,Perf Month %,Perf Quart %,Perf Half %,Perf Year %,Perf YTD %,Beta,ATR,Volatility W %,Volatility M %,SMA20 %,SMA50 %,SMA200 %,50D High %,50D Low %,52W High %,52W Low %,52W Range,All-Time High %,All-Time Low %,RSI,Earnings,IPO Date,Optionable,Shortable,Employees,Change from Open %,Gap %,Recom,"Avg Volume, M",Rel Volume,Volume,Target Price,Prev Close,Open,High,Low,Price,Change %,Single Category,Asset Type,Expense %,Holdings,"AUM, M","Flows 1M, M",Flows% 1M,"Flows 3M, M",Flows% 3M,"Flows YTD, M",Flows% YTD,Return% 1Y,Return% 3Y,Return% 5Y,Tags,Info,"MktCap AUM, M",Rank
Ticker,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
NVDA,1,NVIDIA Corp,"DJIA, NDX, S&P 500",Technology,Semiconductors,USA,NASD,3740520.0,49.38,26.82,1.67,25.19,44.59,69.67,51.91,3.44,2.20,0.03,0.04,6/11/2025,1.16,3.10,1.00,44.35,32.42,91.83,29.57,64.24,69.18,27.60,81.36,86.17,148510.0,76770.0,9.89,1.68,24390.00,23410.00,95.97,4.08,-0.41,66.39,0.60,0.88,0.83,206.80,75.89,115.46,81.82,3.39,2.96,0.12,0.12,70.11,58.03,51.69,3.65,13.45,41.45,9.55,23.64,14.16,2.13,4.01,2.59,2.21,4.65,16.43,18.26,-3.41,61.30,-3.41,76.98,86.62 - 158.71,-3.41,459799.99,65.81,May 28/a,1/22/1999,Yes,Yes,36000.0,-1.92,-1.07,1.38,249.07000,0.85,212161747,173.58,157.99,156.30,157.20,151.49,153.30,-2.97,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-,"Technology, Semiconductors",3740520.0,1
MSFT,2,Microsoft Corporation,"DJIA, NDX, S&P 500",Technology,Software - Infrastructure,USA,NASD,3657180.0,38.02,32.47,2.61,13.54,11.36,45.93,52.72,43.30,10.71,0.67,2.41,8/21/2025,25.42,12.94,3.37,13.50,13.16,18.45,14.55,14.33,13.27,17.88,12.10,14.13,270010.0,96640.0,7.38,2.38,7430.00,7320.00,98.50,1.48,-0.12,73.61,0.68,0.70,2.18,51.17,18.46,33.61,23.24,1.37,1.36,0.29,0.33,69.07,45.23,35.79,0.40,6.88,31.08,12.31,8.66,16.74,1.03,6.85,1.18,1.24,2.50,9.18,16.11,-1.74,38.34,-1.74,42.71,344.79 - 500.76,-1.74,617443.16,69.37,Apr 30/a,3/13/1986,Yes,Yes,228000.0,-0.86,-0.22,1.31,23.44000,0.85,19912468,522.14,497.41,496.31,498.05,490.98,492.05,-1.08,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-,"Technology, Software - Infrastructure",3657180.0,2
AAPL,3,Apple Inc,"DJIA, NDX, S&P 500",Technology,Consumer Electronics,USA,NASD,3103960.0,32.43,26.73,4.23,7.75,46.48,64.00,31.52,4.47,3.25,0.49,1.01,5/12/2025,16.11,6.41,1.42,6.21,8.45,15.41,7.67,8.51,5.08,7.68,-0.36,4.91,400370.0,97290.0,1.39,0.86,14940.00,14920.00,99.88,0.10,-1.28,63.81,-0.17,0.67,1.59,100.23,29.10,138.02,66.93,0.82,0.78,1.18,1.47,46.63,31.81,24.30,3.75,3.47,-6.44,-19.77,-2.93,-17.01,1.20,4.51,2.15,1.93,3.41,2.35,-6.88,-3.14,9.49,-20.10,22.82,169.21 - 260.10,-20.10,326578.80,60.65,May 01/a,12/12/1980,Yes,Yes,164000.0,0.53,0.76,2.00,63.00000,1.24,78245930,228.41,205.17,206.72,210.19,206.14,207.82,1.29,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-,"Technology, Consumer Electronics",3103960.0,3
AMZN,4,Amazon.com Inc,"DJIA, NDX, S&P 500",Consumer Cyclical,Internet Retail,USA,NASD,2340480.0,35.96,30.28,2.07,3.60,7.65,23.82,112.47,28.82,9.25,NaN,NaN,-,0.00,6.13,1.32,12.20,17.36,36.89,17.36,17.86,8.62,62.33,71.88,10.08,650310.0,65940.0,16.38,0.33,10610.00,9490.00,89.45,10.58,-0.32,64.43,0.39,0.65,1.22,61.84,11.23,25.24,15.02,1.05,0.84,0.44,0.49,49.16,11.15,10.14,3.61,7.54,15.87,-2.90,11.43,0.49,1.34,5.08,2.43,2.11,3.16,8.71,7.38,-1.50,33.38,-9.10,45.41,151.61 - 242.52,-9.10,335839.07,62.48,May 01/a,